In [2]:
import json
import datetime
import pandas as pd
import ipaddress
import random
from pymongo import MongoClient
from random import randint

In [3]:
# generate the 10 X 1000000 new database with key words
# "sourceIP","port","first seen","last seen"
# get the process time
start = datetime.datetime.now()
MAX_IP_INT = 4200000000
N=1000000

protocols = ['TCP','UDP','ssh']

for i in range(228):
    netflow = []
    now_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i)
    last_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i) + datetime.timedelta(minutes = randint(0,5))
    for n in range(N):
        IPs = {
        "srcaddr" :  "{}".format(str(ipaddress.ip_address(randint(0, MAX_IP_INT)))),
        "srcaddr" :  randint(0,65535),
        "First"   :  int(now_time.strftime("%Y%m%d%H%M%S")),
        "Last"    :  int(last_time.strftime("%Y%m%d%H%M%S")),
        "protocol":  random.choice(protocols),
        "flows"   :  randint(0,30),
        "packets" :  randint(0,500),
        "bytes"   :  randint(0,1000000),
#        "src_mask":  "{}".format(str(ipaddress.ip_address(randint(0, MAX_IP_INT)))),
#        "hostname":  xyz1.srv.lrz.de
#        "os"      :
#        "Subnet"  :
        }
        netflow.append(IPs)

    json_object = json.dumps(netflow, indent = 3)
    now_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i)
    file_name = 'data/full/' + now_time.strftime("%Y%m%d%H%M%S") + '.json'
    with open(file_name, "w") as outfile:
        outfile.write(json_object)
        outfile.close()
    
end = datetime.datetime.now()
print(end-start)

1:33:12.701285


In [10]:
# Python read all files from a directory in order
import os

f = open("./data/data_name.txt", 'w')
path = "./data/spt/"
files = os.listdir(path)
files.sort()

s = []

for file_name in files:
    if not os.path.isdir(path + file_name):
        f_name = str(file_name)
        print(f_name)
        s.append(f_name)
        f.write(f_name + '\n')
for i in s:
    print(i)
print(s)

20200526125000.json
20200526125525.json
20200526130053.json
20200526130618.json
20200526131145.json
20200526131710.json
20200526132235.json
20200526132759.json
20200526133324.json
20200526133848.json
20200526125000.json
20200526125525.json
20200526130053.json
20200526130618.json
20200526131145.json
20200526131710.json
20200526132235.json
20200526132759.json
20200526133324.json
20200526133848.json
['20200526125000.json', '20200526125525.json', '20200526130053.json', '20200526130618.json', '20200526131145.json', '20200526131710.json', '20200526132235.json', '20200526132759.json', '20200526133324.json', '20200526133848.json']


In [12]:
# 1. Read data into databases, comparison of the process time, plot each database engine process time as a line
# 1.1 Read 10 X 1000000 records into mongodb

#client = MongoClient(port=27017)
#db = client.netflow
read_time = []

for json_name in s:
    client = MongoClient(port=27017)
    db = client.netflow
    start = datetime.datetime.now()
    with open('data/spt/'+json_name) as record:
        new_record = json.load(record)

    db.spt.insert_many(new_record)
    end = datetime.datetime.now()
    read_time_part = end-start
    read_time.append(read_time_part)
    print(read_time_part)
    #db.command({closeAllDatabases:1})
    client.close()

#03client.close()
print(read_time)


0:00:24.074949
0:00:26.847165
0:00:24.762502
0:00:24.678265
0:00:25.482162
0:00:24.905563
0:00:23.906058
0:00:25.365848
0:00:25.478512
0:00:25.829741
[datetime.timedelta(0, 24, 74949), datetime.timedelta(0, 26, 847165), datetime.timedelta(0, 24, 762502), datetime.timedelta(0, 24, 678265), datetime.timedelta(0, 25, 482162), datetime.timedelta(0, 24, 905563), datetime.timedelta(0, 23, 906058), datetime.timedelta(0, 25, 365848), datetime.timedelta(0, 25, 478512), datetime.timedelta(0, 25, 829741)]


In [ ]:
client = MongoClient(port=27017)
db=client.netflow
# Count the total flows in whole database
start = datetime.datetime.now()
total_flows = db.spt.aggregate([
    {
        '$group':{
            '_id': "null",
            'total': { '$sum': "$flows"}
        }
    }
])
for total in total_flows:
    print(total)
    
client.close()    
end = datetime.datetime.now()
print(end-start)

In [ ]:
# generate the 10 X 1000000 new database with key words
# "sourceIP","port","first seen","last seen"
# get the process time
start = datetime.datetime.now()
MAX_IP_INT = 4200000000
N=1000000

protocols = ['TCP','UDP']

for i in range(10):
    netflow = []
    now_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i)
    for n in range(N):
        IPs = {
        "srcaddr" :  "{}".format(str(ipaddress.ip_address(randint(0, MAX_IP_INT)))),
        "srcport" :  randint(0,65535),
        "dstaddr" :  "{}".format(str(ipaddress.ip_address(randint(0, MAX_IP_INT)))),        
        "dstport" :  randint(0,65535),
        "first"   :  int(now_time.strftime("%Y%m%d%H%M%S")),
        "last"    :  int(now_time.strftime("%Y%m%d%H%M%S")),
        "flows"   :  randint(0,30),
        "packets" :  randint(0,500),
        "bytes"   :  randint(0,1000000),
        "protocol":  random.choice(protocols),
        }
        netflow.append(IPs)

    json_object = json.dumps(netflow, indent = 3)
    now_time = datetime.datetime.now() + datetime.timedelta(minutes = 5 * i)
    file_name = 'spt' + now_time.strftime("%Y%m%d%H%M%S") + '.json'
    with open(file_name, "w") as outfile:
        outfile.write(json_object)
        outfile.close()
    
end = datetime.datetime.now()
print(end-start)